In [1]:
import pandas as pd
import sklearn
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import re

### Load the Data

In [2]:
tv=pd.read_csv('TV_Commercials_Combines.csv')

In [3]:
duplicate = tv.duplicated()
print(duplicate.sum())
tv[duplicate]

18063


,1,2,3,4,5,6,7,8,9,10,...,882,924,959,1002,1016,1048,1112,4124,4125,Label
111622,34,7.221330,3.191483,23.518745,11.492748,0.012884,0.010939,0.070129,0.042020,3479.684814,...,NaN,NaN,0.017241,NaN,NaN,NaN,NaN,0.340551,0.263356,1
111623,26,6.180094,3.015362,32.932747,18.120752,0.013149,0.010831,0.075000,0.050982,3362.287354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.969324,0.623752,1
111624,25,2.462173,1.541003,20.039402,18.033579,0.015869,0.011657,0.062625,0.042195,3317.864746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.148112,0.780920,1
111625,37,6.194136,2.658522,22.864201,13.588374,0.017141,0.012120,0.078041,0.072270,3491.640137,...,NaN,0.015625,0.023438,NaN,NaN,NaN,0.007812,0.746379,0.403130,1
111626,25,4.367200,2.063856,31.797680,14.305157,0.013647,0.013007,0.107625,0.080645,3327.662109,...,NaN,NaN,0.025000,NaN,NaN,NaN,0.087500,0.572578,0.588162,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129680,30,2.808389,1.475269,9.408182,3.851741,0.011282,0.010490,0.116250,0.075856,3421.109131,...,NaN,0.070000,0.040000,0.020000,0.030000,0.020000,NaN,0.840342,0.015348,1
129681,32,4.830225,1.974286,7.508844,3.180844,0.015507,0.014104,0.127734,0.071251,3479.608887,...,NaN,0.018519,0.009259,NaN,0.009259,0.027778,NaN,0.342186,0.555814,1
129682,32,5.308545,2.534434,10.520031,4.518093,0.017690,0.013312,0.104492,0.063117,3484.652588,...,NaN,0.009259,0.009259,NaN,NaN,0.009259,NaN,0.667208,0.821335,1
129683,42,0.426690,0.192054,2.405270,1.100260,0.013657,0.008062,0.106696,0.046419,3616.494629,...,NaN,NaN,NaN,0.013514,NaN,NaN,NaN,0.200665,0.354387,1


In [4]:
tv.drop_duplicates(keep=False,inplace=True) 

In [5]:
duplicate = tv.duplicated()
print(duplicate.sum())
tv[duplicate]

0


,1,2,3,4,5,6,7,8,9,10,...,882,924,959,1002,1016,1048,1112,4124,4125,Label


### Seleksi Features dengan menghapus feature yang memiliki  nilai null > 25%

In [6]:
print(tv.isnull().sum()/tv.shape[0])

1        0.000000
2        0.000000
3        0.000000
4        0.000000
5        0.000000
           ...   
1048     0.575519
1112     0.902799
4124     0.000000
4125     0.000000
Label    0.000000
Length: 205, dtype: float64


In [7]:
df_dataset=tv.loc[:,((tv.isnull().sum()/tv.shape[0])*100)<25]

Berdasarkan proses feature selection, dihasilkan 100 columns. Columns ini yang akan digunakan untuk membangun model klasifikasi

In [8]:
df_dataset.shape

(93559, 98)

### Memetakan nomor kolom menjadi nama kolom

In [9]:
MotionDist=[2,3]
FrameDiff=[4,5]
ShortTime=[6,7]
ZCR=[8,9]
SpectralCentroid=[10,11]
SpectralRoll=[12,13]
SpectralFlux=[14,15]
FundamentalFreq=[16,17]
EdgeChange=[4124,4125]
MotionDistbins=[*range(18, 59)]
FrameDiffbins=[*range(59, 92)]
TextAreaDistbins=[*range(92, 123)]
BagAudioWordsbins=[*range(123,4124)] 
l_MotionDist=list()

for m in range(0,len(MotionDistbins)):
    l_MotionDist.append("Motion Distribution bin_"+ str(MotionDistbins[m]))

l_FrameDiff=list()
for f in range(0,len(FrameDiffbins)):
    l_FrameDiff.append("Frame Difference bin_"+ str(FrameDiffbins[f]))
    

l_TextArea=list()
for t in range(0,len(TextAreaDistbins)):
    l_TextArea.append("Text Area Distribution bin_"+ str(TextAreaDistbins[t]))

l_BagAudioWords=list()
for b in range(0,len(BagAudioWordsbins)):
    l_BagAudioWords.append("Bag Audio Words bin_"+ str(BagAudioWordsbins[b]))
    
df_dict={ 1: "Shot Length" ,
        MotionDist[0] : "Motion Distribution Mean",
        MotionDist[1] : "Motion Distribution Variance",
        FrameDiff[0]: 'Frame Difference Distribution Mean',
        FrameDiff[1]: 'Frame Difference Distribution Variance',
        ShortTime[0] : 'Short time energy Mean',
        ShortTime[1] : 'Short time energy Variance',
        ZCR[0] : 'ZCR Mean',
        ZCR[1] : 'ZCR Variance',
        SpectralCentroid[0]:'SpectralCentroid Mean',
        SpectralCentroid[1]:'SpectralCentroid Variance',
        SpectralRoll[0]: 'Spectral Roll off Mean',
        SpectralRoll[1]: 'Spectral Roll off Variance',
        SpectralFlux[0]:'Spectral Flux Mean',
        SpectralFlux[1]:'Spectral Flux Variance',
        FundamentalFreq[0]: 'Fundamental Frequecy Mean',
        FundamentalFreq[1]:'Fundamental Frequecy Variance',
        EdgeChange[0]:'Edge Change Ratio Mean',
        EdgeChange[1]:'Edge Change Ratio Variance',       
}

for i in range(0,len(MotionDistbins)):
    df_dict[MotionDistbins[i]] = l_MotionDist[i]
    
for j in range(0,len(FrameDiffbins)):
    df_dict[FrameDiffbins[j]] = l_FrameDiff[j]
    
for k in range(0,len(TextAreaDistbins)):
    df_dict[TextAreaDistbins[k]] = l_TextArea[k]

for p in range(0,len(BagAudioWordsbins)):
    df_dict[BagAudioWordsbins[p]] = l_BagAudioWords[p]

In [10]:
namedict=df_dict
cols=df_dataset.columns
cols=[int(i) for i in cols[:-1]]
colnames=[namedict[i] for i in cols]
colnames.append('Label')

In [11]:
df_dataset.columns=colnames

In [12]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93559 entries, 0 to 111621
Data columns (total 98 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Shot Length                             93559 non-null  int64  
 1   Motion Distribution Mean                93559 non-null  float64
 2   Motion Distribution Variance            93559 non-null  float64
 3   Frame Difference Distribution Mean      93559 non-null  float64
 4   Frame Difference Distribution Variance  93559 non-null  float64
 5   Short time energy Mean                  93559 non-null  float64
 6   Short time energy Variance              93559 non-null  float64
 7   ZCR Mean                                93559 non-null  float64
 8   ZCR Variance                            93559 non-null  float64
 9   SpectralCentroid Mean                   93559 non-null  float64
 10  SpectralCentroid Variance               93559 non-null  f

### Dataset mengandung nilai null sehingga data perlu dibersihkan dengan mengisi nilai Mean masing-masing kolom pada data null

In [13]:
print(df_dataset.isnull().values.any())

True


In [14]:
# menghitung means dan kemudian mengisi nilai null
column_means = df_dataset.mean()
clean_dataset = df_dataset.fillna(column_means)

In [15]:
# periksa kembali apakah masih ada data yang missing
print(clean_dataset.isnull().values.any())

False


In [16]:
clean_dataset

,Shot Length,Motion Distribution Mean,Motion Distribution Variance,Frame Difference Distribution Mean,Frame Difference Distribution Variance,Short time energy Mean,Short time energy Variance,ZCR Mean,ZCR Variance,SpectralCentroid Mean,...,Text Area Distribution bin_118,Text Area Distribution bin_119,Bag Audio Words bin_491,Bag Audio Words bin_572,Bag Audio Words bin_573,Bag Audio Words bin_623,Bag Audio Words bin_816,Edge Change Ratio Mean,Edge Change Ratio Variance,Label
0,123,1.316440,1.516003,5.605905,5.346760,0.013233,0.010729,0.091743,0.050768,3808.067871,...,0.251688,0.176144,0.069915,0.141949,0.103814,0.050847,0.036017,0.422334,0.663918,1
1,124,0.966079,0.546420,4.046537,3.190973,0.008338,0.011490,0.075504,0.065841,3466.266113,...,0.206926,0.051774,0.075630,0.012605,0.037815,0.086134,0.023109,0.332664,0.766184,1
2,109,2.035407,0.571643,9.551406,5.803685,0.015189,0.014294,0.094209,0.044991,3798.196533,...,0.185767,0.038852,0.069712,0.038462,0.088942,0.139423,0.052885,0.346674,0.225022,1
3,86,3.206008,0.786326,10.092709,2.693058,0.013962,0.011039,0.092042,0.043756,3761.712402,...,0.165600,0.003983,0.135802,0.012346,0.086420,0.212963,0.077160,0.993323,0.840083,1
4,76,3.135861,0.896346,10.348035,2.651010,0.020914,0.012061,0.108018,0.052617,3784.488037,...,0.159495,0.000066,0.021127,0.007042,0.154930,0.151408,0.017606,0.341520,0.710470,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111617,84,0.571141,0.429302,5.601481,4.493654,0.016669,0.010648,0.113988,0.045117,3736.336670,...,0.066948,0.051338,0.041139,0.072785,0.303797,0.110759,0.069620,0.672047,0.647189,1
111618,25,1.242366,0.772255,19.877289,11.755177,0.016593,0.010652,0.097625,0.052185,3305.738037,...,0.101005,0.073604,0.050000,0.075000,0.537500,0.100000,0.012500,0.180709,0.103151,1
111619,64,0.566330,0.443887,5.739406,6.426752,0.015055,0.011919,0.098682,0.048214,3637.395508,...,0.085835,0.066839,0.076271,0.114407,0.161017,0.063559,0.008475,0.202583,0.046392,1
111620,71,0.815467,0.401193,3.989804,1.596751,0.019880,0.011590,0.109419,0.045566,3706.726074,...,0.064876,0.053495,0.037879,0.064394,0.352273,0.125000,0.045455,0.977861,0.410044,1


In [17]:
from sklearn.preprocessing import LabelEncoder
dataset=clean_dataset.apply(LabelEncoder().fit_transform)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93559 entries, 0 to 111621
Data columns (total 98 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   Shot Length                             93559 non-null  int64
 1   Motion Distribution Mean                93559 non-null  int64
 2   Motion Distribution Variance            93559 non-null  int64
 3   Frame Difference Distribution Mean      93559 non-null  int64
 4   Frame Difference Distribution Variance  93559 non-null  int64
 5   Short time energy Mean                  93559 non-null  int64
 6   Short time energy Variance              93559 non-null  int64
 7   ZCR Mean                                93559 non-null  int64
 8   ZCR Variance                            93559 non-null  int64
 9   SpectralCentroid Mean                   93559 non-null  int64
 10  SpectralCentroid Variance               93559 non-null  int64
 11  Spectral Roll 

In [18]:
import numpy as np
dataset = dataset.replace('NaN', 0)

In [18]:
X = dataset.drop(['Label'], axis = 1)
y = dataset.Label

In [19]:
# Rescale the data
X=dataset.drop('Label',axis=1) #Predictors
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX = scaler.fit_transform(X)

# Convert X back to a Pandas DataFrame, for convenience
X = pd.DataFrame(rescaledX, columns=X.columns)

In [20]:
# # Rescale the data

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0,1))
# rescaledX = scaler.fit_transform(X)

# # Convert X back to a Pandas DataFrame, for convenience
# X = pd.DataFrame(rescaledX, columns=X.columns)

### Split Into Test and Training Sets

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### Scaling Data

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Generating model for K=5

In [22]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
# y_pred = knn.predict(X_test)
# print("Result for the test set is : {}".format(y_pred))
# print("Score on training set is : {:.2f}%".format(knn.score(X_train, y_train)*100))
# print("Score on test set is : {:.2f}%".format(knn.score(X_test, y_test)*100))


##### accuracy score for data train

In [23]:
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_train)
accuracy = accuracy_score(y_train, y_pred);
print ('accuracy:%0.2f%%'%(accuracy*100))

accuracy:92.21%


##### accuracy score for data test

In [24]:
from sklearn.metrics import accuracy_score
y_pred_test = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test);
print ('accuracy:%0.2f%%'%(accuracy*100))

accuracy:88.32%


In [25]:
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [27]:
from sklearn.metrics import accuracy_score
print("Recall score", recall_score(y_test, y_pred_test, average='macro'))
print("Precision score", precision_score(y_test, y_pred_test, average='macro'))
print ("CONFUSION MATRIX", confusion_matrix(y_test, y_pred_test))
print ("Accuracy Score", accuracy_score(y_test, y_pred_test))

Recall score 0.8655864723707363
Precision score 0.8813414113162916
CONFUSION MATRIX [[ 8292  2102]
 [ 1177 16497]]
Accuracy Score 0.8831765711842667


In [ ]:
new = pd.read_excel('./new_data.xlsx')

In [ ]:
# y_pred = knn.predict(new)

### Generating model for K=7

In [28]:
from sklearn.neighbors import KNeighborsClassifier
knn2 = KNeighborsClassifier(n_neighbors=7)

knn2.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
# y_pred = knn.predict(X_test)
# print("Result for the test set is : {}".format(y_pred))
# print("Score on training set is : {:.2f}%".format(knn.score(X_train, y_train)*100))
# print("Score on test set is : {:.2f}%".format(knn.score(X_test, y_test)*100))

##### accuracy score for data train

In [30]:
from sklearn.metrics import accuracy_score
pred = knn2.predict(X_train)
accuracy = accuracy_score(y_train, pred);
print ('accuracy:%0.2f%%'%(accuracy*100))

accuracy:91.04%


##### accuracy score for data test

In [31]:
from sklearn.metrics import accuracy_score
pred_test = knn2.predict(X_test)
accuracy = accuracy_score(y_test, pred_test);
print ('accuracy:%0.2f%%'%(accuracy*100))

accuracy:88.13%


In [32]:
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [33]:
from sklearn.metrics import accuracy_score
print("Recall score", recall_score(y_test, pred_test, average='macro'))
print("Precision score", precision_score(y_test, pred_test, average='macro'))
print ("CONFUSION MATRIX", confusion_matrix(y_test, pred_test))
print ("Accuracy Score", accuracy_score(y_test, pred_test))

Recall score 0.8631869651995197
Precision score 0.8795099161214308
CONFUSION MATRIX [[ 8248  2146]
 [ 1187 16487]]
Accuracy Score 0.8812526720820864


In [39]:
new = pd.read_excel('./new_data.xlsx')

In [42]:
pred_test = knn2.predict(new)

In [43]:
pred_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)